## PowerPlants class

In [161]:
import pandas as pd
import os

class PowerPlants(object):
    def __init__(self):
        self.database_file = 'database.csv'

    def _load_database(self) -> pd.DataFrame:
        if os.path.exists(self.database_file):
            df = pd.read_csv(self.database_file)

            # handling duplicate 'Date'/'date' columns
            if 'date' in df.columns and 'Date' in df.columns:
                parsed_date = pd.to_datetime(df['date'], dayfirst=False, errors='coerce').dt.normalize()
                parsed_Date = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce').dt.normalize()
                if parsed_date.equals(parsed_Date):
                    df.drop(columns='Date', inplace=True)
                else:
                    raise ValueError("Conflict: 'Date' and 'date' columns have different data.")
            
            df.columns = df.columns.str.strip()

            # ensuring standard datetime format
          
            df['updatetime'] = pd.to_datetime(df['updatetime'])

            return df
        else:
            return pd.DataFrame()

    def _prepare_new_data(self, df: pd.DataFrame) -> pd.DataFrame:
        df.columns = df.columns.str.strip()
        df.dropna(how='all', inplace=True)
        df.fillna(0, inplace=True)

    
        if 'updatedby' not in df.columns:
            df['updatedby'] = 'petroineos'
        if 'updatetime' not in df.columns:
            df['updatetime'] = pd.Timestamp.now()

        # ensuring column names are consistent
        if 'Date' in df.columns:
            df.rename(columns={'Date': 'date'}, inplace=True)

        df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce').dt.normalize()
        df['updatetime'] = pd.to_datetime(df['updatetime'])

        country_lookup = {
        'GB': 'Great Britain',
        'FR': 'France'
        }
        if 'Country' in df.columns:
            df['country'] = df['Country'].map(country_lookup)

        return df

    def load_new_data_from_file(self, file_path: str) -> pd.DataFrame:
        df = pd.read_csv(file_path)
        return self._prepare_new_data(df)

    def save_new_data(self, input_data: pd.DataFrame):
        db_df = self._load_database()

        # ensure input columns match database columns
        if not db_df.empty:
            for col in db_df.columns:
                if col not in input_data.columns:
                    input_data[col] = None
            input_data = input_data[db_df.columns]

        combined = pd.concat([db_df, input_data], ignore_index=True)
        combined.sort_values('updatetime', ascending=False, inplace=True)
        combined = combined.drop_duplicates(subset=['SiteName', 'date'], keep='first')
        
        desired_order = ['date', 'country', 'SiteName', 'Technology', 'updatedby', 'updatetime', 'Volume']
        combined = combined[[col for col in desired_order if col in combined.columns]]
        combined.to_csv(self.database_file, index=False)

    def get_data_from_database(self) -> pd.DataFrame:
       
        df = self._load_database()
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
        return df


    def aggregate_data_to_monthly(self) -> pd.DataFrame:
        df = self.get_data_from_database()
        df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()

        monthly_agg = df.groupby(['month','SiteName' ])['Volume'].agg(
        average_volume='mean',
        min_volume='min',
        max_volume='max'
        ).reset_index()

        wide_format = monthly_agg.pivot(index='month', columns='SiteName')

        sites = sorted(wide_format.columns.get_level_values(1).unique())
        stats = ['average_volume', 'min_volume', 'max_volume']
        ordered = pd.DataFrame({'date': wide_format.index})
        for site in sites:
            for stat in stats:
                if (stat, site) in wide_format.columns:
                    output_label = stat.replace('_', ' ').capitalize()
                    colname = f"{site} {output_label}"
                    ordered[colname] = wide_format[(stat, site)].values
        return ordered


        
    def aggregate_data_to_country(self) -> pd.DataFrame:
        df = self.get_data_from_database()

        country_agg = df.groupby(['country', 'Technology']).agg({
            'Volume': 'sum'
        }).reset_index()

        country_agg.rename(columns={'Volume': 'total_volume'}, inplace=True)
        return country_agg



## Testing

In [162]:
pp = PowerPlants()
new_data = pp.load_new_data_from_file('wind_plants.csv')
pp.save_new_data(new_data)
new_data = pp.load_new_data_from_file('gas_plants.csv')
pp.save_new_data(new_data)
new_data = pp.load_new_data_from_file('gas_fr_plants.csv')
pp.save_new_data(new_data)

In [163]:
from IPython.display import display

print("Final database:")
df = pp.get_data_from_database()
display(df)


Final database:


,date,country,SiteName,Technology,updatedby,updatetime,Volume
0,2025-04-25,France,Blenod-5,Gas,petroineos,2025-06-05 16:15:29.739945,6035.000000
1,2024-04-29,France,Blenod-5,Gas,petroineos,2025-06-05 16:15:29.739945,6016.000000
2,2024-06-01,France,Blenod-5,Gas,petroineos,2025-06-05 16:15:29.739945,4252.000000
3,2024-06-02,France,Blenod-5,Gas,petroineos,2025-06-05 16:15:29.739945,5211.000000
4,2024-06-03,France,Blenod-5,Gas,petroineos,2025-06-05 16:15:29.739945,3853.000000
...,...,...,...,...,...,...,...
5277,NaT,Great Britain,Hornsea-1,Wind,petroineos,2025-06-04 18:52:18.357859,512.296429
5278,NaT,Great Britain,Hornsea-2,Wind,petroineos,2025-06-04 18:52:18.357859,929.595893
5279,NaT,Great Britain,Hornsea-2,Wind,petroineos,2025-06-04 18:52:18.357859,689.112870
5280,NaT,Great Britain,Hornsea-1,Wind,petroineos,2025-06-04 18:52:18.357859,140.587671


In [164]:
# Aggregation by month 
print("Monthly aggregation:")
display(pp.aggregate_data_to_monthly())

Monthly aggregation:


,date,Blenod-5 Average volume,Blenod-5 Min volume,Blenod-5 Max volume,Hornsea-1 Average volume,Hornsea-1 Min volume,Hornsea-1 Max volume,Hornsea-2 Average volume,Hornsea-2 Min volume,Hornsea-2 Max volume,Pembroke-1 Average volume,Pembroke-1 Min volume,Pembroke-1 Max volume,Pembroke-2 Average volume,Pembroke-2 Min volume,Pembroke-2 Max volume
0,2024-01-01,5198.806452,3295.0,6890.0,502.454543,44.937929,900.648050,447.211079,85.603428,892.386165,7092.903226,5390.0,8905.0,7195.096774,5074.0,8941.0
1,2024-02-01,4889.896552,3489.0,6895.0,535.210353,9.973714,980.763621,437.902354,35.733954,994.618512,6738.620690,5222.0,8970.0,6878.551724,5051.0,8955.0
2,2024-03-01,4752.774194,3122.0,6399.0,432.654207,34.240333,864.461415,592.607677,122.681263,979.909946,7499.322581,5196.0,8963.0,6605.806452,5094.0,8685.0
3,2024-04-01,4988.466667,3235.0,6970.0,561.504570,67.202179,995.133071,504.877587,37.514079,999.165096,6946.566667,5251.0,8941.0,7142.633333,5167.0,8845.0
4,2024-05-01,5012.032258,3059.0,6898.0,431.650067,14.319806,913.805516,536.694996,16.694314,962.658133,7211.290323,5009.0,8938.0,6759.806452,5039.0,8888.0
5,2024-06-01,4928.666667,3083.0,6870.0,384.724509,16.042355,852.995514,387.852090,1.797161,979.952783,7003.300000,5001.0,8978.0,7368.933333,5247.0,8934.0
6,2024-07-01,4933.677419,3115.0,6680.0,471.688183,8.136290,944.258261,510.198669,13.206362,850.887204,7052.580645,5077.0,8983.0,7057.483871,5017.0,8981.0
7,2024-08-01,5296.258065,3104.0,6973.0,627.744512,6.695016,988.355244,474.907802,37.231691,999.000373,7203.032258,5049.0,8977.0,7455.225806,5262.0,8978.0
8,2024-09-01,4858.066667,3110.0,6922.0,474.785256,24.260159,945.447225,466.750343,80.138479,992.225257,7382.333333,5149.0,8925.0,6596.333333,5009.0,8931.0
9,2024-10-01,4326.741935,0.0,6973.0,524.710537,75.917168,990.985647,520.673636,43.367492,988.350146,7202.129032,5162.0,8923.0,6711.483871,5101.0,8909.0


In [171]:
# Aggregation by country
print("Country-tech aggregation:")
display(pp.aggregate_data_to_country())

Country-tech aggregation:


,country,Technology,total_volume
0,France,Gas,7.138749e+06
1,Great Britain,Gas,1.353625e+07
2,Great Britain,Wind,9.507131e+05
